# Project 3

In this project, you will perform a logistic regression on the admissions data we've been working with in projects 1 and 2.

In [8]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import statsmodels.api as sm
import pylab as pl
import numpy as np


In [9]:
df_raw = pd.read_csv("../assets/admissions.csv")
df = df_raw.dropna() 
print df.head()

   admit    gre   gpa  prestige
0      0  380.0  3.61       3.0
1      1  660.0  3.67       3.0
2      1  800.0  4.00       1.0
3      1  640.0  3.19       4.0
4      0  520.0  2.93       4.0


In [10]:
df.prestige = df.prestige.astype(int) 
df.head()

/Users/luisfelipevelez/anaconda/lib/python2.7/site-packages/pandas/core/generic.py:2701: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


,admit,gre,gpa,prestige
0,0,380.0,3.61,3
1,1,660.0,3.67,3
2,1,800.0,4.00,1
3,1,640.0,3.19,4
4,0,520.0,2.93,4


## Part 1. Frequency Tables

#### 1. Let's create a frequency table of our variables

In [11]:
# frequency table for prestige and whether or not someone was admitted
pd.crosstab(df.prestige, df.admit, margins=True)

admit,0,1,All
prestige,,,
1,28,33,61
2,95,53,148
3,93,28,121
4,55,12,67
All,271,126,397


## Part 2. Return of dummy variables

#### 2.1 Create class or dummy variables for prestige 

In [12]:
#seasons = pd.get_dummies(sub_data.season, prefix='season')
presdums = pd.get_dummies(df.prestige,prefix='prestige')
presdums.head()

,prestige_1,prestige_2,prestige_3,prestige_4
0,0.0,0.0,1.0,0.0
1,0.0,0.0,1.0,0.0
2,1.0,0.0,0.0,0.0
3,0.0,0.0,0.0,1.0
4,0.0,0.0,0.0,1.0


#### 2.2 When modeling our class variables, how many do we need? 



Answer: for "prestige" class, we'd need three variables.

## Part 3. Hand calculating odds ratios

Develop your intuition about expected outcomes by hand calculating odds ratios.

In [13]:
cols_to_keep = ['admit', 'gre', 'gpa']
handCalc = df[cols_to_keep].join(presdums.ix[:, 'prestige_1':])
print handCalc.head()

   admit    gre   gpa  prestige_1  prestige_2  prestige_3  prestige_4
0      0  380.0  3.61         0.0         0.0         1.0         0.0
1      1  660.0  3.67         0.0         0.0         1.0         0.0
2      1  800.0  4.00         1.0         0.0         0.0         0.0
3      1  640.0  3.19         0.0         0.0         0.0         1.0
4      0  520.0  2.93         0.0         0.0         0.0         1.0


In [14]:
#crosstab prestige 1 admission 
# frequency table cutting prestige and whether or not someone was admitted
pd.crosstab(handCalc.prestige_1, handCalc.admit, margins=True)

admit,0,1,All
prestige_1,,,
0.0,243,93,336
1.0,28,33,61
All,271,126,397


#### 3.1 Use the cross tab above to calculate the odds of being admitted to grad school if you attended a #1 ranked college

p(admit=1 if prestige_1=1)=(33/61)=54.1%

#### 3.2 Now calculate the odds of admission if you did not attend a #1 ranked college

p(admit=1 if prestige_1=0)=(93/336)=27.7%

#### 3.3 Calculate the odds ratio

OR = [p(admit=1 if prestige_1=1) / p(admit=1 if prestige_1=0)] = (54.1% / 27.7%) = 1.95x

#### 3.4 Write this finding in a sentenance: 

Answer: Out of all students admited into grad school, there are ~1.95 students that attended at "prestige 1" school for each 1 student admited that did not attend a "prestige 1" school

#### 3.5 Print the cross tab for prestige_4

In [15]:
pd.crosstab(handCalc.prestige_4, handCalc.admit, margins=True)

admit,0,1,All
prestige_4,,,
0.0,216,114,330
1.0,55,12,67
All,271,126,397


#### 3.6 Calculate the OR 

OR = [p(admit=1 if prestige_4=1) / p(admit=1 if prestige_4=0)] = (17.9% / 34.5%) = 0.52x

#### 3.7 Write this finding in a sentence

Answer: Out of all students admitted into grad school, there are ~0.52 students that attended a "prestige 4" school for each 1 student admitted that did not attend a "prestige 4" school

## Part 4. Analysis

In [16]:
# create a clean data frame for the regression
cols_to_keep = ['admit', 'gre', 'gpa']
data = df[cols_to_keep].join(presdums.ix[:, 'prestige_2':])
print data.head()

   admit    gre   gpa  prestige_2  prestige_3  prestige_4
0      0  380.0  3.61         0.0         1.0         0.0
1      1  660.0  3.67         0.0         1.0         0.0
2      1  800.0  4.00         0.0         0.0         0.0
3      1  640.0  3.19         0.0         0.0         1.0
4      0  520.0  2.93         0.0         0.0         1.0


We're going to add a constant term for our Logistic Regression. The statsmodels function we're going to be using requires that intercepts/constants are specified explicitly.

In [17]:
# manually add the intercept
data['intercept'] = 1.0
y=data['intercept']
print data.head()

   admit    gre   gpa  prestige_2  prestige_3  prestige_4  intercept
0      0  380.0  3.61         0.0         1.0         0.0        1.0
1      1  660.0  3.67         0.0         1.0         0.0        1.0
2      1  800.0  4.00         0.0         0.0         0.0        1.0
3      1  640.0  3.19         0.0         0.0         1.0        1.0
4      0  520.0  2.93         0.0         0.0         1.0        1.0


#### 4.1 Set the covariates to a variable called train_cols

In [54]:
train_cols = data.columns[1:]
print train_cols

Index([u'gre', u'gpa', u'prestige_2', u'prestige_3', u'prestige_4',
       u'intercept'],
      dtype='object')


#### 4.2 Fit the model

In [26]:
logit = sm.Logit(data['admit'], train_cols)

In [27]:
result = logit.fit()

Optimization terminated successfully.
         Current function value: 0.589121
         Iterations 5


#### 4.3 Print the summary results

In [28]:
print result.summary()

                           Logit Regression Results                           
Dep. Variable:                  admit   No. Observations:                  397
Model:                          Logit   Df Residuals:                      392
Method:                           MLE   Df Model:                            4
Date:                Wed, 19 Oct 2016   Pseudo R-squ.:                 0.05722
Time:                        19:14:39   Log-Likelihood:                -233.88
converged:                       True   LL-Null:                       -248.08
                                        LLR p-value:                 1.039e-05
                 coef    std err          z      P>|z|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
gre            0.0014      0.001      1.308      0.191        -0.001     0.003
gpa           -0.1323      0.195     -0.680      0.497        -0.514     0.249
prestige_2    -0.9562      0.302     -3.171      0.0

#### 4.4 Calculate the odds ratios of the coeffiencents and their 95% CI intervals

hint 1: np.exp(X)

hint 2: conf['OR'] = params
        
           conf.columns = ['2.5%', '97.5%', 'OR']

In [29]:
print np.exp(result.params) #odds ratio

gre           1.001368
gpa           0.876073
prestige_2    0.384342
prestige_3    0.214918
prestige_4    0.154135
dtype: float64


In [33]:
params = result.params
conf = result.conf_int()
conf['OR'] = params
conf.columns = ['2.5%', '97.5%', 'OR']
print np.exp(conf)#odds

                2.5%     97.5%        OR
gre         0.999320  1.003420  1.001368
gpa         0.598303  1.282800  0.876073
prestige_2  0.212826  0.694082  0.384342
prestige_3  0.112055  0.412207  0.214918
prestige_4  0.070176  0.338540  0.154135


#### 4.5 Interpret the OR of Prestige_2

Answer: Within the universe of students admitted, there are 0.38 people from a prestige 2 unis for every 1 there is of any other university. It is 0.38 times as likely to find a student admitted from a prestige 2 uni, as it is to find a student of any other prestige rank.   

#### 4.6 Interpret the OR of GPA

Answer: 

## Part 5: Predicted probablities


As a way of evaluating our classifier, we're going to recreate the dataset with every logical combination of input values. This will allow us to see how the predicted probability of admission increases/decreases across different variables. First we're going to generate the combinations using a helper function called cartesian (above).

We're going to use np.linspace to create a range of values for "gre" and "gpa". This creates a range of linearly spaced values from a specified min and maximum value--in our case just the min/max observed values.

In [34]:
def cartesian(arrays, out=None):
    """
    Generate a cartesian product of input arrays.
    Parameters
    ----------
    arrays : list of array-like
        1-D arrays to form the cartesian product of.
    out : ndarray
        Array to place the cartesian product in.
    Returns
    -------
    out : ndarray
        2-D array of shape (M, len(arrays)) containing cartesian products
        formed of input arrays.
    Examples
    --------
    >>> cartesian(([1, 2, 3], [4, 5], [6, 7]))
    array([[1, 4, 6],
           [1, 4, 7],
           [1, 5, 6],
           [1, 5, 7],
           [2, 4, 6],
           [2, 4, 7],
           [2, 5, 6],
           [2, 5, 7],
           [3, 4, 6],
           [3, 4, 7],
           [3, 5, 6],
           [3, 5, 7]])
    """

    arrays = [np.asarray(x) for x in arrays]
    dtype = arrays[0].dtype

    n = np.prod([x.size for x in arrays])
    if out is None:
        out = np.zeros([n, len(arrays)], dtype=dtype)

    m = n / arrays[0].size
    out[:,0] = np.repeat(arrays[0], m)
    if arrays[1:]:
        cartesian(arrays[1:], out=out[0:m,1:])
        for j in xrange(1, arrays[0].size):
            out[j*m:(j+1)*m,1:] = out[0:m,1:]
    return out

In [69]:
# instead of generating all possible values of GRE and GPA, we're going
# to use an evenly spaced range of 10 values from the min to the max 
gres = np.linspace(data['gre'].min(), data['gre'].max(), 10)
print gres
# array([ 220.        ,  284.44444444,  348.88888889,  413.33333333,
#         477.77777778,  542.22222222,  606.66666667,  671.11111111,
#         735.55555556,  800.        ])
gpas = np.linspace(data['gpa'].min(), data['gpa'].max(), 10)
print gpas
# array([ 2.26      ,  2.45333333,  2.64666667,  2.84      ,  3.03333333,
#         3.22666667,  3.42      ,  3.61333333,  3.80666667,  4.        ])


# enumerate all possibilities
combos = pd.DataFrame(cartesian([gres, gpas, [1, 2, 3, 4], [1.]]))

[ 220.          284.44444444  348.88888889  413.33333333  477.77777778
  542.22222222  606.66666667  671.11111111  735.55555556  800.        ]
[ 2.26        2.45333333  2.64666667  2.84        3.03333333  3.22666667
  3.42        3.61333333  3.80666667  4.        ]


#### 5.1 Recreate the dummy variables

In [70]:
# recreate the dummy variables
combos.columns = ['gre', 'gpa', 'prestige', 'intercept']
presdums2 = pd.get_dummies(combos.prestige,prefix='prestige')
# keep only what we need for making predictions
cols_to_keep = ['gre', 'gpa', 'prestige', 'intercept']
combos = combos[cols_to_keep].join(presdums2.ix[:, 'prestige_2':])

#### 5.2 Make predictions on the enumerated dataset

In [ ]:
result.predict(combos[train_cols])

#### 5.3 Interpret findings for the last 4 observations

Answer: 

## Bonus

Plot the probability of being admitted into graduate school, stratified by GPA and GRE score.